In [7]:
#!/usr/bin/env python3
"""
Generate convergence-test directories for cubic III–V semiconductors (zincblende form)
using Materials Project data and local VASP pseudopotentials (NERSC, Jupyter version).
"""

# === Environment setup (Jupyter-friendly) ===
import os
os.environ["PMG_VASP_PSP_DIR"] = "/global/homes/x/xiaozha/pp/"

from pymatgen.ext.matproj import MPRester
from pymatgen.io.vasp.sets import MPStaticSet
from pymatgen.io.vasp.inputs import Potcar, Kpoints

Potcar.POTCAR_DIR = os.environ["PMG_VASP_PSP_DIR"]

# === USER SETTINGS ===
root_dir = "./convergence_tests"
III = ["B", "Al", "Ga", "In"]
V = ["N", "P", "As", "Sb"]
kgrids = [(4, 4, 4), (6, 6, 6), (8, 8, 8), (10, 10, 10), (12,12,12)]
encuts = [300, 400, 500, 600, 700]

mpr = MPRester("RXi5d8z5XEhPXDtNTHxuiYP5FTX5ObEN")

COMMON_INCAR = {
    "NSW": 0,
    "ISMEAR": 0,
    "SIGMA": 0.0001,
    "PREC": "Normal",
    "ISYM": 2,
    "KPAR": 4,
    "EDIFF": 1e-6
}

# === Utility helpers ===
def safe_mkdir(path):
    os.makedirs(path, exist_ok=True)

def write_slurm_script(path, jobname):
    """Create a small submit.slurm file under each folder."""
    slurm_script = f"""#!/bin/bash -l
#SBATCH -J {jobname}
#SBATCH -A <YOUR_ALLOCATION>
#SBATCH -C cpu
#SBATCH -q regular
#SBATCH -t 01:00:00
#SBATCH -N 1
#SBATCH --ntasks-per-node=64
#SBATCH --cpus-per-task=1

module load vasp/6.4.3
srun vasp_std > vasp.out
"""
    with open(os.path.join(path, "submit.slurm"), "w") as f:
        f.write(slurm_script)

# === MAIN LOOP ===
for a in III:
    for b in V:
        formula = f"{a}{b}"
        print(f"\n=== {formula} ===")

        # Query Materials Project
        docs = mpr.summary.search(
            formula=formula,
            fields=["material_id", "symmetry", "structure", "energy_above_hull"],
        )
        if not docs:
            print(f"⚠️ No entry found for {formula}")
            continue

        # Pick cubic zincblende (SG 216 = F-43m)
        structure = None
        for doc in docs:
            sgnum = getattr(doc.symmetry, "number", None)
            if sgnum == 216:
                structure = doc.structure
                print(f"✅ Found zincblende form (mp-id: {doc.material_id})")
                break
        if structure is None:
            print(f"⚠️ No cubic zincblende form found for {formula}")
            continue

        # === Base directory ===
        mat_dir = os.path.join(root_dir, formula)
        safe_mkdir(mat_dir)

        # ===== K-POINT CONVERGENCE =====
        conv_k_dir = os.path.join(mat_dir, "conv_kpt")
        safe_mkdir(conv_k_dir)
        for kgrid in kgrids:
            subdir = os.path.join(conv_k_dir, f"k{kgrid[0]}x{kgrid[1]}x{kgrid[2]}")
            safe_mkdir(subdir)

            incar = COMMON_INCAR | {"ENCUT": 500}
            vasp_set = MPStaticSet(structure, user_incar_settings=incar)
            vasp_set.write_input(subdir)

            # Overwrite KPOINTS explicitly
            kpoints = Kpoints.monkhorst_automatic(kpts=kgrid, shift=(0, 0, 0))
            kpoints.comment = f"Gamma-centered {kgrid[0]}x{kgrid[1]}x{kgrid[2]}"
            kpoints.style = "Gamma"
            kpoints.write_file(os.path.join(subdir, "KPOINTS"))

            write_slurm_script(subdir, f"{formula}_k{kgrid[0]}")

        # ===== ENCUT CONVERGENCE =====
        conv_e_dir = os.path.join(mat_dir, "conv_cutoff")
        safe_mkdir(conv_e_dir)
        for e in encuts:
            subdir = os.path.join(conv_e_dir, f"E{e}")
            safe_mkdir(subdir)

            incar = COMMON_INCAR | {"ENCUT": e}
            vasp_set = MPStaticSet(structure, user_incar_settings=incar)
            vasp_set.write_input(subdir)
            write_slurm_script(subdir, f"{formula}_E{e}")

print("\n✅ All convergence-test folders created successfully.")



=== BN ===


/tmp/ipykernel_273046/3699898774.py:65: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  docs = mpr.summary.search(
Retrieving SummaryDoc documents: 100%|██████████| 27/27 [00:00<00:00, 647121.19it/s]


✅ Found zincblende form (mp-id: mp-1639)

=== BP ===


Retrieving SummaryDoc documents: 100%|██████████| 2/2 [00:00<00:00, 45100.04it/s]


✅ Found zincblende form (mp-id: mp-1479)

=== BAs ===


...

✅ Found zincblende form (mp-id: mp-20351)

=== InAs ===


Retrieving SummaryDoc documents: 100%|██████████| 5/5 [00:00<00:00, 94042.69it/s]


✅ Found zincblende form (mp-id: mp-20305)

=== InSb ===


Retrieving SummaryDoc documents: 100%|██████████| 11/11 [00:00<00:00, 288358.40it/s]


✅ Found zincblende form (mp-id: mp-20012)

✅ All convergence-test folders created successfully.
